In [ ]:
import requests
from PIL import Image
import numpy as np
import io
import base64
import matplotlib.pyplot as plt

# Step 1: Fetch the cameraman image from the specified URL
# URL: https://raw.githubusercontent.com/scikit-image/scikit-image/main/skimage/data/camera.png
url = "https://raw.githubusercontent.com/scikit-image/scikit-image/main/skimage/data/camera.png"
response = requests.get(url)
if response.status_code != 200:
    raise Exception("Failed to fetch the image from the provided URL")

# Step 2: Load the image and convert to grayscale
# Open the image from the response content
image = Image.open(io.BytesIO(response.content))

# Convert to grayscale ('L' mode in PIL) for consistency
grayscale_image = image.convert('L')

# Step 3: Convert the grayscale image to a numpy array
# Creates a matrix where each element is a pixel intensity (0-255)
image_array = np.array(grayscale_image)

# Step 4: Ensure the image matrix is square for eigenvalue computation
# Crop to the smallest dimension to make it square
height, width = image_array.shape
size = min(height, width)
square_image = image_array[:size, :size]

# Step 5: Create a symmetric matrix to ensure real eigenvalues
# Compute A^T A to get a symmetric positive semi-definite matrix
# This guarantees real, non-negative eigenvalues
symmetric_matrix = np.dot(square_image.T, square_image)

# Step 6: Compute eigenvalues and eigenvectors of the symmetric matrix
# Use np.linalg.eigh for symmetric matrices to ensure real eigenvalues
eigenvalues, eigenvectors = np.linalg.eigh(symmetric_matrix)

# Step 7: Ensure eigenvalues are real and sort in descending order
# np.linalg.eigh guarantees real eigenvalues; cast to real for safety
eigenvalues = np.real(eigenvalues)

# Sort eigenvalues and eigenvectors in descending order
# Get indices that would sort eigenvalues in descending order
sort_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvalues = eigenvalues[sort_indices]
sorted_eigenvectors = eigenvectors[:, sort_indices]

# Step 8: Prepare the grayscale image for display as base64
# Convert the grayscale image to a PNG in memory
buffer = io.BytesIO()
grayscale_image.save(buffer, format="PNG")
img_str = base64.b64encode(buffer.getvalue()).decode()

# Step 9: Create a scree plot of the sorted eigenvalues
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(sorted_eigenvalues) + 1), sorted_eigenvalues, 'b-', marker='o', markersize=4)
plt.title('Scree Plot of Eigenvalues (Descending Order)')
plt.xlabel('Index')
plt.ylabel('Eigenvalue Magnitude')
plt.grid(True)
plt.yscale('log')  # Log scale for better visualization of large eigenvalue range

# Save the plot to a BytesIO buffer as PNG
plot_buffer = io.BytesIO()
plt.savefig(plot_buffer, format='png', bbox_inches='tight')
plt.close()
plot_str = base64.b64encode(plot_buffer.getvalue()).decode()

# Step 10: Print a summary of results
print(f"Original image dimensions: {height}x{width}")
print(f"Square matrix size used: {size}x{size}")
print(f"Number of eigenvalues: {len(sorted_eigenvalues)}")
print("First five eigenvalues (real numbers):")
print(sorted_eigenvalues[:5])
print("Eigenvalues as array (first 10 for brevity):")
print(sorted_eigenvalues[:10])
print("Shape of sorted eigenvectors matrix:", sorted_eigenvectors.shape)

